In [37]:
%reset -f

import sys
import time
from svector import svector
import operator
from collections import defaultdict

def read_from(textfile):
    for line in open(textfile):
        label, words = line.strip().split("\t")
        yield (1 if label=="+" else -1, words.split())

def make_vector(words):
    v = svector()
    for word in words:
        v[word] += 1
    v['<bias>'] = -1
    return v
    
def test(devfile, model):
    tot, err = 0, 0
    for i, (label, words) in enumerate(devfile, 1): # note 1...|D|
        err += label * (model.dot(make_vector(words))) <= 0
    return err/i  # i is |D| now
            
def train(trainfile, devfile, epochs=5):
    t = time.time()
    best_err = 1.
    wordDict, oneWordCountSet = defaultdict(int), set()
    model = svector()
    avgModel = svector()
    c=0
#     print('initial model: {}'.format(model))

    for label,words in trainfile:
        for word in words:
            wordDict[word] += 1
#     print('len of wordDict: {}'.format(len(wordDict)))
    
    for word in wordDict:
        if wordDict[word] == 1:
            oneWordCountSet.add(word)
#     print('wordDict: {}'.format(len(oneWordCountSet)))
                        
    for it in range(1, epochs+1):
        updates = 0
        for i, (label, words) in enumerate(trainfile, 1): # label is +1 or -1
            sent = make_vector([word for word in words if word not in oneWordCountSet])
            
            if label * (model.dot(sent)) <= 0:
                updates += 1
                model += label * sent
                avgModel += c*label*sent
            c += 1
#         print('len of sent: {} len of model: {} and avgmodel: {}'.format(len(sent), len(model), len(avgModel)))
            
        dev_err = test(devfile, model)
        dev_err_avg = test(devfile, c*model - avgModel)
        
        best_err = min(best_err, dev_err_avg)
        print("epoch %d, update %.1f%%, dev %.1f%%" % (it, updates / i * 100, dev_err_avg * 100))
    print("best dev err %.1f%%, |w|=%d, time: %.1f secs" % (best_err * 100, len(model), time.time() - t))
    
    
    return model, avgModel

if __name__ == "__main__":
    trainfile = list(read_from('train.txt'))
    devfile = list(read_from('dev.txt'))
    
    model, avgModel = train(trainfile, devfile, 10)
#     sortedWeightVectors = sorted(avgModel, key=avgModel.get)
#     print('len of model: {}'.format(len(model)))
#     print('------------------------')
#     print('Most positive features: {}'.format(sortedWeightVectors[:20]))
#     print('------------------------')
#     print('Most negative features: {}'.format(sortedWeightVectors[-20:]))

len of wordDict: 15805
wordDict: 7381
epoch 1, update 39.0%, dev 31.6%
epoch 2, update 26.4%, dev 27.5%
epoch 3, update 22.8%, dev 26.8%
epoch 4, update 18.8%, dev 26.6%
epoch 5, update 17.2%, dev 25.9%
epoch 6, update 14.8%, dev 26.5%
epoch 7, update 13.2%, dev 27.0%
epoch 8, update 12.7%, dev 26.7%
epoch 9, update 11.4%, dev 26.6%
epoch 10, update 10.6%, dev 26.2%
best dev err 25.9%, |w|=9974, time: 1.8 secs
